
# ============================================
# LangGraph Mini-Lab 4 — Chatbot Agent with Llama (TinyLlama, no API keys)
# ============================================
**Author:** Dr. Dasha Trofimova

This notebook builds a **chatbot-style agent** using **LangGraph** for control flow and an **open-source LLM** (TinyLlama) via `transformers` — mirroring the structure of `Agent_Bot.py`, but **no OpenAI keys** required.



### What you'll learn
- Wrap a Hugging Face chat model as a simple LLM function
- Maintain **conversation state** (messages) inside a LangGraph `State`
- Add a **system prompt**, keep the **last N turns**, and run a REPL chat loop
- (Optional) add a tiny **calculator tool** and route via a decision node


In [ ]:

!pip install -q langgraph transformers accelerate sentencepiece --upgrade


In [ ]:

import torch
from transformers import pipeline

DEVICE = 0 if torch.cuda.is_available() else -1
print("cuda available?", torch.cuda.is_available())

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

gen = pipeline(
    task="text-generation",
    model=model_name,
    tokenizer=model_name,
    device_map="auto" if DEVICE != -1 else None,
    torch_dtype="auto",
    max_new_tokens=256,
    temperature=0.2,
    do_sample=True
)

print("Loaded:", model_name)


In [ ]:

SYSTEM_PROMPT = (
    "You are a helpful teaching assistant. Keep answers concise, concrete, and cite equations or steps when relevant. "
    "If you don't know, say so briefly."
)

def format_prompt(messages, system=SYSTEM_PROMPT, max_rounds=6):
    sys = f"System: {system}\n\n"
    ua = [m for m in messages if m["role"] in ("user","assistant")]
    ua = ua[-(2*max_rounds):]
    lines = []
    for m in ua:
        role = "User" if m["role"] == "user" else "Assistant"
        lines.append(f"{role}: {m['content']}")
    return sys + "\n".join(lines) + "\nAssistant:"

def llm_chat(messages):
    prompt = format_prompt(messages)
    out = gen(prompt)[0]["generated_text"]
    if "Assistant:" in out:
        reply = out.split("Assistant:", 1)[-1].strip()
    else:
        reply = out.strip()
    return reply


In [ ]:

from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict

class ChatState(TypedDict):
    messages: List[Dict[str,str]]
    last_reply: str

NODE_DESC = {
    "chat_turn": "Calls the TinyLlama model with the current chat history and appends assistant reply.",
}

def trace(name):
    def deco(fn):
        def wrap(state: ChatState) -> ChatState:
            print(f"\n▶ Node: {name} — {NODE_DESC.get(name,'')}")
            out = fn(state)
            return out
        wrap.__doc__ = NODE_DESC.get(name,"")
        return wrap
    return deco

@trace("chat_turn")
def chat_turn(state: ChatState) -> ChatState:
    reply = llm_chat(state["messages"])
    new_msgs = state["messages"] + [{"role":"assistant","content":reply}]
    return {"messages": new_msgs, "last_reply": reply}

g = StateGraph(ChatState)
g.add_node("chat_turn", chat_turn)
g.add_edge("chat_turn", END)
g.set_entry_point("chat_turn")
chatbot = g.compile()
print("Chatbot agent ready.")


In [ ]:

def ask_once(chatbot, messages, user_text):
    state = {"messages": messages + [{"role":"user","content":user_text}], "last_reply": ""}
    out = chatbot.invoke(state)
    return out["messages"], out["last_reply"]

messages = [{"role":"system","content": SYSTEM_PROMPT}]
messages, reply = ask_once(chatbot, messages, "Hi! What is attention in transformers in one sentence?")
print("Assistant:", reply)



## Optional: add a calculator tool via routing
We add a **decide** node that routes to either **calc** or **chat_turn**.
- Queries with numbers/operators go to the calculator
- Everything else goes to the LLM


In [ ]:

import re
from typing import Literal, TypedDict, List, Dict

def is_math_query(q: str) -> bool:
    return bool(re.fullmatch(r"[0-9\s+\-*/().]+", q))

def calc_eval(q: str) -> str:
    try:
        return str(eval(q, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Calc error: {e}"

class AgentState(TypedDict):
    messages: List[Dict[str,str]]
    route: Literal["calc","chat"]
    last_reply: str

NODE_DESC2 = {
    "decide": "Router: if query looks like math, route to 'calc' else 'chat'.",
    "calc": "Calculator: returns the numeric result.",
    "chat": "LLM chat node (TinyLlama)."
}

def trace2(name):
    def deco(fn):
        def wrap(state: AgentState) -> AgentState:
            print(f"\n▶ Node: {name} — {NODE_DESC2.get(name,'')}")
            out = fn(state)
            return out
        wrap.__doc__ = NODE_DESC2.get(name,"")
        return wrap
    return deco

@trace2("decide")
def decide(state: AgentState) -> AgentState:
    user_msgs = [m for m in state["messages"] if m["role"] == "user"]
    last_q = user_msgs[-1]["content"] if user_msgs else ""
    route = "calc" if is_math_query(last_q) else "chat"
    return {"messages": state["messages"], "route": route, "last_reply": state["last_reply"]}

@trace2("calc")
def calc_node(state: AgentState) -> AgentState:
    user_msgs = [m for m in state["messages"] if m["role"] == "user"]
    last_q = user_msgs[-1]["content"] if user_msgs else ""
    ans = calc_eval(last_q)
    msgs = state["messages"] + [{"role":"assistant","content": ans}]
    return {"messages": msgs, "route": state["route"], "last_reply": ans}

@trace2("chat")
def chat_node(state: AgentState) -> AgentState:
    reply = llm_chat(state["messages"])
    msgs = state["messages"] + [{"role":"assistant","content": reply}]
    return {"messages": msgs, "route": state["route"], "last_reply": reply}

from langgraph.graph import StateGraph, END
rg = StateGraph(AgentState)
rg.add_node("decide", decide)
rg.add_node("calc", calc_node)
rg.add_node("chat", chat_node)

def router(state: AgentState):
    return state["route"]

rg.add_conditional_edges("decide", router, {"calc":"calc", "chat":"chat"})
rg.add_edge("calc", END); rg.add_edge("chat", END)
rg.set_entry_point("decide")
agent = rg.compile()

msgs = [{"role":"system","content": SYSTEM_PROMPT}]
def run_agent(msgs, user_text):
    state = {"messages": msgs + [{"role":"user","content": user_text}], "route": "chat", "last_reply": ""}
    out = agent.invoke(state)
    return out["messages"], out["last_reply"]

msgs, ans1 = run_agent(msgs, "2*(3+4)")
print("Assistant (calc):", ans1)
msgs, ans2 = run_agent(msgs, "Explain attention in 2 bullets.")
print("Assistant (chat):", ans2[:200], "...")



## Agent Anatomy & Graph
- **State keys**: `messages`, `route`, `last_reply`
- **Nodes**: `decide` → `calc`/`chat` → END


In [ ]:

!apt-get -qq update && apt-get -qq install -y graphviz > /dev/null
from graphviz import Digraph

dot = Digraph(comment="Chat Agent", format="png")
dot.attr(rankdir="LR", bgcolor="white")
dot.node("decide","decide()", shape="diamond", style="rounded,filled", fillcolor="#F3F4F6")
dot.node("calc","calc()", shape="box", style="rounded,filled", fillcolor="#DBEAFE")
dot.node("chat","chat()", shape="box", style="rounded,filled", fillcolor="#DCFCE7")
dot.edge("decide","calc", label="looks like math")
dot.edge("decide","chat", label="otherwise")
display(dot)



## Interactive REPL
Run this to chat in the cell. Type `exit` to stop.


In [ ]:

def repl():
    msgs = [{"role":"system","content": SYSTEM_PROMPT}]
    while True:
        try:
            q = input("\nYou: ")
        except EOFError:
            break
        if q.strip().lower() in {"exit","quit"}:
            break
        msgs, ans = run_agent(msgs, q)
        print("Assistant:", ans[:1200])
    print("Bye!")

# Uncomment to use:
# repl()
